In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import DataFrame
from functools import reduce
import findspark
import boto3
import os
findspark.init()

In [3]:
os.environ["AWS_ACCESS_KEY_ID"] = "IUESG4784AFDSGUIGHFSED"
os.environ["AWS_SECRET_ACCESS_KEY"] = "umgD/iCR+JHFAEGVHGFDSBJHrtejusdz3JVYCdxkeYzz"
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

In [4]:
aws_access_key_id = os.environ.get("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.environ.get("AWS_SECRET_ACCESS_KEY")
region_name = os.environ.get("AWS_DEFAULT_REGION")

In [5]:
spark = (SparkSession.builder
        .appName("ColabS3Upload")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3a.access.key", aws_access_key_id)
        .config("spark.hadoop.fs.s3a.secret.key", aws_secret_access_key)
        .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com")
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4")
        .getOrCreate()
      )

In [6]:
def read_dataset(path:str, comma:bool=True)-> DataFrame:
  sep = ',' if comma else ';'
  df = spark.read.csv(path, sep=sep, header=True)
  return df

In [7]:
def create_bucket(bucket_name:str):
  s3 = boto3.client(
      's3',
      aws_access_key_id= aws_access_key_id,
      aws_secret_access_key= aws_secret_access_key,
      region_name=region_name
  )
  s3.create_bucket(Bucket=bucket_name)

In [10]:
bucket_name = 'project-pnad-covid-bronze-layer'
create_bucket(bucket_name)

In [8]:
path_2020_09 = '/content/PNAD_COVID_092020.csv'
path_2020_10 = '/content/PNAD_COVID_102020.csv'
path_2020_11 = '/content/PNAD_COVID_112020.csv'
path_dict = '/content/dicionario_PNAD_COVID_112020_20220621.csv'

# Reading project datasets
df_2020_09 = read_dataset(path = path_2020_09)
df_2020_10 = read_dataset(path = path_2020_10)
df_2020_11 = read_dataset(path = path_2020_11)
df_dict = read_dataset(path = path_dict, comma = False)

In [11]:
df_2020_09.write.mode("overwrite").parquet(f"s3a://{bucket_name}/PNAD_COVID_092020_parquet/")
df_2020_10.write.mode("overwrite").parquet(f"s3a://{bucket_name}/PNAD_COVID_102020_parquet/")
df_2020_11.write.mode("overwrite").parquet(f"s3a://{bucket_name}/PNAD_COVID_112020_parquet/")
df_dict.write.mode("overwrite").parquet(f"s3a://{bucket_name}/dicionario_PNAD_COVID_parquet/")

In [12]:
df_dict.write.mode("overwrite").parquet(f"s3a://{bucket_name}/dicionario_PNAD_COVID_parquet/")